<h1>Projet de mathématiques</h1>

<p>Ce notebook contient les fonctions nécessaires et les éventuels résultats de nos recherches pour la partie deux du projet de mathématiques.</p>

Groupe 21 :
<ul><li>DUONG Ngo</li><li>DIENG Seydina</li><li>REIGNER Thibaud</li></ul>

# Modèle de Black-Scholes

In [71]:
# Bibliothèques numpy et scipy pour faciliter la gestion des données et du calcul numérique
import scipy as sp
import scipy.stats
import numpy as np

# Bibliothèque et sous-modules pour la représentation 2D/3D des données
from matplotlib import cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d


# Pour ignorer les warnings de division par zéro pour le calcul d'erreur relative
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

## Pricer par la méthode de Monte-Carlo

In [3]:
#Question n°12

def pricer_MC (n, s, r, sigma, T, f):
    #On crée d'abord la suite (Ksi) composée de n variables aléatoires iid, suivant N(0,1):
    ksi = scipy.stats.norm.rvs(0, 1, n)
    #On initialise la valeur de retour :
    res = 0
    #Et on applique la formule donnée:
    for i in range(n):
        res += np.exp(-r*T)*f(s*np.exp((r - ((sigma**2)/2))*T + sigma*np.sqrt(T)*ksi[i]))
    res = res / n
    return res

In [96]:
#Question n°13

f = lambda x: max(x - 100, 0)

#Attention le calcul peut-être un peu long (environ 20 secondes)
res =[]
val_n=[]
for k in range(1,11):
    val = pricer_MC(k*10**5, 100, 0.01, 0.1, 1, f)
    val_n.append(k*10**5)
    res.append(val)

plt.scatter(val_n, res, color='blue')
plt.xlabel("$n = 10^5k$")
plt.ylabel("Prix déterminé par le pricerMC")
plt.title("Evolution du prix par le pricerMC en fonction de $n$")
#plt.savefig("evol_pricerMC.png")
plt.show()

## Pricer par formule fermée

In [5]:
#Question n°15

def put_BS(s, r, sigma, T, K):
    #On calcule d1 et d2:
    d1 = (1/(sigma*np.sqrt(T)))*(np.log(s/K) + (r + (sigma**2)/2)*T)
    d2 =  d1 - sigma*np.sqrt(T)

    #On définit ensuite la fonction F
    def F(x):
        return scipy.stats.norm.cdf(x, 0, 1)
        
    #On réalise finalement le calcul :
    res = -s*F(-d1) + K*np.exp(-r*T)*F(-d2)
    return res

In [6]:
#Question n°16

#Test du put_BS

put_BS(100,0.01,0.1,1,90)

0.5815000751362422

In [99]:
#Question n°17

#Tracé de Pricer_MC et put_BS

f = lambda x: max(90 - x, 0)

res =[]
val_n=[]
for k in range(1,11):
    val = pricer_MC(k*10**5, 100, 0.01, 0.1, 1, f)
    val_n.append(k*10**5)
    res.append(val)

# On trace le Pricer_MC

plt.plot(val_n, res, color='blue', label='pricer2')
plt.scatter(val_n, res, color='orange', s=13) 

#On trace put_BS
putBS= put_BS(100, 0.01, 0.1, 1, 90)
plt.plot(val_n, putBS*np.array([1 for k in range(1, 11)]), color='red', label='put_BS')

plt.xlabel("$n = 10^5k$")
plt.ylabel("Prix par le pricer_MC")
plt.title("Comparaison du Put_BS et du pricer_MC")
plt.legend()
plt.show()


In [100]:
#Question n°18

fig = plt.figure(figsize=(10,8))
trois_axes = fig.add_subplot(111, projection='3d')

T={1,2,3,4,6,12}
for k in range(1,11):
    for t in T:
        val = put_BS(20*k, 0.01, 0.1, 1/t, 100)
        trois_axes.scatter(20*k, 1/t, val, color="blue", s=20)

plt.title("Prix de l'option avec Put_BS")
trois_axes.set_xlabel("$s$")
trois_axes.set_ylabel("$T$")
trois_axes.set_zlabel("Put_BS")


plt.show()

## Convergence des Prix

In [14]:
#On redonne la fonction Pricer_2
 
#fonction pricer_2 :

def pricer_2(N, rn, hn, bn, s, f):
    qn=(rn-bn) / (hn-bn)
    #On modélise l'arbre des v_k par une grosse matrice (n+1) * (n+1)
    V=np.zeros((N+1, N+1))
    #Etape 1 : On va construire l'arbre par la fin puisqu'on connait les valeurs possibles de V_n=f(S_tN)
    for i in range(0, N+1):
        V[i][N]= f(s*((1+hn)**(N-i)) * (1+bn)**i)
    #Etape 2 : On remonte l'arbre, l'espérance est alors facile à calculer, cf le rapport
    for j in range(N-1, -1, -1):
        for i in range(0, j+1):
            V[i][j]=(1/(1+rn)) * (qn * V[i][j+1] + (1-qn) * V[i+1][j+1])

    return V[0][0]

In [101]:
#Question n°19

f = lambda x: max(100 - x, 0)

res = []
val_n = []
for k in range(1,101):
    #On calcule rn, hn et bn
    rn = (0.03*1)/(10*k)
    hn = (1 + rn)*np.exp(0.2*np.sqrt(1/(10*k))) - 1
    bn = (1 + rn)*np.exp(- 0.2*np.sqrt(1/(10*k))) - 1

    #On en déduit la valeur de pricer_2
    val = pricer_2(10*k,rn, hn, bn, 100, f)
    res.append(val)
    val_n.append(10*k)

plt.plot(val_n, res, color='blue', label='pricer2')
plt.scatter(val_n, res, color='orange', s=13)  

#On trace put_BS
plt.plot(val_n, put_BS(100, 0.03, 0.2, 1, 100)*np.array([1 for k in range(1, 101)]), color='green', label='put_BS')

plt.legend()
plt.xlabel("$N$")
plt.ylabel("Valeur de pricer2")
plt.title("Prix de l'option via le Pricer 2 par rapport à Put_BS en fonction de $N$")

plt.show()

# EDP de Black-Scholes

In [20]:
# On posera pour cette question :
K = 0.9
r = 0.015
sigma = 0.21
T = 1
x_min = np.log(0.4)
x_max = np.log(2)
N = 100


## Méthode des différences finies explicite

In [87]:
#Déclarations utiles

#On pose :
M = 100


# Création de la matrice des p(t_m, x_j)
P = np.zeros((M, N ))


# Discrétisation de l'espace-temps
X_liste = np.linspace(x_min, x_max, N)
T_liste = np.linspace(0, T, M)

# Déclaration des pas spatial et temporel
h = X_liste[1]
dt = T_liste[1]

# Conditions initiales
for x in range(1,N-1):
    P[0][x] = max(K - np.exp(X_liste[x]), 0)

for t in range(M):
    P[t][0] = K*np.exp(-r*T_liste[t]) - np.exp(x_min)
    P[t][-1] = 0





In [88]:
#On calcule les coefficients a[i] obtenus après avoir injectés les équations du schéma explicite
#dans l'EDP:

a1 = dt*((1/dt)-r-(sigma/h)**2)
a2 = dt*((1/2)*(sigma/h)**2+(r-(sigma**2/2))*(1/(2*h)))
a3 = dt*((1/2)*(sigma/h)**2-(r-(sigma**2/2))*(1/(2*h)))

for m in range(1, M):
    for n in range(1,N-2):
        P[m][n] = a1*P[m-1][n] + a2*P[m-1][n+1] + a3*P[m-1][n-1]

#print(P[-1])

In [102]:
# Affichage de l'évolution du PUT en fonction de x
plt.plot(X_liste, P[-1, :], color="blue", label="Put_BS")
plt.plot(X_liste, np.fmax(K - np.exp(X_liste), 0), color = "red", label = "payoff")

# Configurations
plt.xlabel("Valeurs de $x$")
plt.ylabel("Valeur du Put_BS de l'option")
plt.title("METHODE EXPLICITE:\nEvolution de la valeur du Put_BS sur l'espace")
plt.legend()
plt.show()

In [103]:
## Calcul de l'erreur relative

err_absol =  np.absolute(P[-1] - np.fmax(K - np.exp(X_liste), 0))
err_relat = err_absol/np.fmax(K - np.exp(X_liste), 0)

# Affichage de l'évolution de l'erreur relative en fonction de x
plt.scatter(X_liste, err_absol, color="blue", label="Différence Finie Explicite")
plt.title("Erreur relative en fonction de l'espace pour la méthode explicite")
plt.legend()
plt.xlabel("Valeurs de $x$")
plt.ylabel("Erreur relative réalisée")
plt.show()

## Méthode des différences finies implicites

In [91]:
#Déclarations utiles

#On pose :
M = 100


# Création de la matrice des p(t_m, x_j)
P = np.zeros((M, N ))


# Discrétisation de l'espace-temps
X_liste = np.linspace(x_min, x_max, N)
T_liste = np.linspace(0, T, M)

# Déclaration des pas spatial et temporel
h = X_liste[1]
dt = T_liste[1]

# Conditions initiales
for x in range(1,N-1):
    P[0][x] = max(K - np.exp(X_liste[x]), 0)

for t in range(M):
    P[t][0] = K*np.exp(-r*T_liste[t]) - np.exp(x_min)
    P[t][-1] = 0

#On va travailler avec des matrices
